# Pricing

In this project, we will compare six different ways to estimate the price of an option from a simple example, a European call :

$$
\boxed{C(0,K) = \mathbb{E}\!\left[ e^{-rT} (S_T - K)_+ \right]}
$$

**Numerical application :**
- Stock : $S_0 = 100$
- Strike : $K = 100$ (at the money at time $t=0$)
- Implied vol : $\sigma = 20\%$
- risk free rate : $r = 2\%$
- maturity : $T=1$ (years)

In [1]:
# Parameter
S0 = 100
K = 100
sigma = 0.2
r = 0.02
T=1

## Sommaire :

* [**1.Direct formula : log-normal distribution (Black Scholes)**](#0)

* [**2.Monte-Carlo estimation**](#1)
    * [2.1. Exact law](#1_1)
    * [2.2. Euler scheme](#1_2)
    * [2.3. Milstein scheme](#1_3)

* [**3.PDE**](#2)




In [2]:
# Modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time
from math import log, sqrt, exp

<a id='0'></a>
## 1. Black–Scholes Formula

Let $(\Omega, \mathcal{F}, \mathbb{P})$ be a probability space, $(W_t)_{t \geq 0}$ a Brownian motion under $\mathbb{P}$, and $(S_t)_{t \geq 0}$ the stock price process.

Let $\mathbb{Q}$ be an equivalent probability measure to $\mathbb{P}$, called the **risk–neutral measure**.

---

>**Assumptions**

Under $\mathbb{Q}$, the dynamics of $S_t$ are given by:  

$$
\frac{dS_t}{S_t} = r \, dt + \sigma \, dW_t,
$$

which implies

$$
S_t = S_0 \exp\!\left( \left(r - \tfrac{1}{2}\sigma^2\right)t + \sigma W_t \right).
$$

---

>**Black–Scholes Formula**

The price of a European call option with maturity $T$ and strike $K$ is

$$
C(0,K) = \mathbb{E}_{\mathbb{Q}}\!\left[ e^{-rT} (S_T - K)_+ \right].
$$

Equivalently,  

$$
C(0,K) 
= e^{-rT} \, \mathbb{E}_{\mathbb{Q}}\!\left[ S_T \, \mathbf{1}_{\{S_T > K\}} \right]
- K e^{-rT} \, \mathbb{Q}(S_T > K).
$$

<br>
<br>

- $ \mathbb{Q}(S_T > K) = \mathbb{Q}\!\left( - \frac{W_T}{\sqrt{T}} < \frac{1}{\sigma \sqrt{T}} \left( \ln\!\left(\tfrac{S_0}{K}\right) + \left(r - \tfrac{1}{2}\sigma^2\right)T \right) \right)  = \Phi(d_2) $ where $\;\Phi(\cdot)$ the cumulative distribution function of the standard normal law.

<br>


- $e^{-rT}\,\mathbb{E}_{\mathbb{Q}}[S_T\mathbf{1}_{\{S_T>K\}}]
= S_0\,\mathbb{E}_{\mathbb{Q}}\!\big[\exp(-\tfrac12\sigma^2T+\sigma W_T)\mathbf{1}_{\{S_T>K\}}\big]$; define Radon-Nikodyn mesure by : $\frac{d\tilde{\mathbb{Q}}}{d\mathbb{Q}}\big|_{\mathcal{F}_T}=\exp(-\tfrac12\sigma^2T+\sigma W_T)$ (Dolean Dade exponential, martingale of expectation $1$) so by Girsanov $\tilde W_t:=W_t-\sigma t$ is BM under $\tilde{\mathbb{Q}}$; hence $=S_0\,\tilde{\mathbb{Q}}(S_T>K)$; with $S_T=S_0\exp((r+\tfrac12\sigma^2)T+\sigma\tilde W_T)$ we get $\tilde{\mathbb{Q}}(S_T>K)=\tilde{\mathbb{Q}}\!\big(\frac{\tilde W_T}{\sqrt T}>\frac{\ln(K/S_0)-(r+\tfrac12\sigma^2)T}{\sigma\sqrt T}\big)=\Phi(d_1)$, where $d_1=\frac{\ln(S_0/K)+(r+\tfrac12\sigma^2)T}{\sigma\sqrt T}$.

Finally :

$$\boxed{C(0,K) = S_0 \,\Phi(d_1) - K e^{-rT}\,\Phi(d_2)}$$


In [3]:
def black_scholes_call(S0: float, K: float, sigma: float, r: float, T: float) -> float:
    """
    Compute the Black–Scholes price of a European call option.

    Parameters
    ----------
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    sigma : float
        Volatility of the underlying (annualized).
    r : float
        Risk-free interest rate (annualized, continuously compounded).
    T : float
        Time to maturity in years.

    Returns
    -------
    float
        Black–Scholes price of the European call option.
    """

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Application
call_BS = black_scholes_call(S0, K, sigma, r, T)
print('Prix de l\'option théorique :', call_BS)

Prix de l'option théorique : 8.916037278572539


<a id='1'></a>
## 2. Monte-Carlo

In [4]:
# Monte carlo parameters
N = 1000
steps = 1000

## 2.1 Exact Law of $(S_t)_{t\geq0}$

Under $\mathbb{Q}$, the dynamic of $S$ is :

$$
\frac{dS_t}{S_t} = r \, dt + \sigma \, dW_t
$$

>**Solution of SDE :** 

$$
d(\ln(S_t)) = \frac{1}{S_t} dS_t - \frac{1}{2} \frac{1}{S_t^2} (dS_t)^2
\implies d(\ln(S_t)) = \frac{1}{S_t} \big(S_t (r  \, dt + \sigma \, dW_t)\big) - \frac{1}{2} \sigma^2 dt
\implies d(\ln(S_t)) = \left(r  - \frac{1}{2} \sigma^2\right) dt + \sigma dW_t
$$

$$
\ln(S_t) = \ln(S_0) + \left(r  - \frac{1}{2} \sigma^2\right)t + \sigma W_t
\implies S_t = S_0 \exp\left(\left(r  - \frac{1}{2} \sigma^2\right)t + \sigma W_t\right)
$$


$$
\implies S_{t_k} = S_{t_{k-1}} \exp\left(\left(r  - \frac{1}{2} \sigma^2\right)(t_k - t_{k-1}) + \sigma \left(W_{t_k} - W_{t_{k-1}}\right)\right)
$$



$$
\implies
\boxed{ \ln \Big(\tfrac{S_{t_k}}{S_{t_{k-1}}}\Big) \,\big|\, \mathcal{F}_s 
\sim \mathcal{N}\!\Big( \big(r-\tfrac12\sigma^2\big)(t_k - t_{k-1}),\; \sigma^2(t_k - t_{k-1}) \Big) }
$$

---


In [5]:
def MC1_call(S0: float, K: float, sigma: float, r: float, T: float, N: int, steps: int) -> float:
    """
    Monte Carlo pricing of a European call option under GBM with exact law.

    Parameters
    ----------
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    sigma : float
        Volatility of the underlying (annualized).
    r : float
        Risk-free interest rate (annualized, continuously compounded).
    T : float
        Time to maturity in years.
    N : int
        Number of simulated paths.
    steps : int
        Number of time steps per path.

    Returns
    -------
    float
        Monte Carlo estimate of the European call price.
    """
    dt = T / steps

    # Draw standard normals: shape (N, steps)
    Z = np.random.randn(N, steps)

    # Log-returns per step
    dlogS = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z

    # Sum over time to get log S_T for each path
    log_ST = np.log(S0) + dlogS.cumsum(axis=1)[:, -1]

    ST = np.exp(log_ST)

    # Discounted payoff
    payoffs = np.exp(-r * T) * np.maximum(ST - K, 0.0)

    return float(payoffs.mean())

# Application
call_MC1 = MC1_call(S0, K, sigma, r, T, N, steps)
print('Prix de l\'option MC1 :', call_MC1)

Prix de l'option MC1 : 8.574843703369035


## 2.2 Exact Law + Control Variates

We want to estimate the expected discounted payoff of a European call option under the risk-neutral measure:

$$
C_t(T,K) = \mathbb{E}_{\mathbb{Q}}\!\left[ e^{-r(T-t)} (S_T - K)_+ \right]
$$

---

**Monte Carlo Estimator**

Let us denote by
$$
A_i = e^{-r(T-t)} (S_T^{(i)} - K)_+
$$
the discounted payoff obtained from the $ i^{th} $ simulated path $ S_T^{(i)} $.

Then the **Monte Carlo (MC) estimator** of $ C_t(T,K) $ based on $ n $ simulations is:

$$
\hat{C}_{MC} = \frac{1}{n} \sum_{i=1}^{n} A_i.
$$

This estimator is **unbiased** and **consistent**:
$$
\mathbb{E}[\hat{C}_{MC}] = C_t(T,K), \quad \hat{C}_{MC} \xrightarrow{a.s.} C_t(T,K) \text{ as } n \to \infty.
$$

---

**Introduction of Control Variates coeff**

To reduce the variance of the Monte Carlo estimator, we introduce a **control variate** $X_i$ correlated with $A_i$ and whose expectation is known analytically.

Lets consider :

$$
X_i = e^{-r(T-t)} S_T^{(i)},
$$
because:
$$
\mathbb{E}[X_i] = e^{-r(T-t)} \mathbb{E}_{\mathbb{Q}}[S_T] = e^{-r(T-t)} S_t e^{r(T-t)} = S_t.
$$

We define the **adjusted sample**:
$$
A_i^{(b)} = A_i - b(X_i - \mathbb{E}[X_i]) = A_i - b(X_i - S_t).
$$

and the corresponding **control variate estimator**:
$$
\hat{C}_{CV}(b) = \frac{1}{n} \sum_{i=1}^{n} A_i^{(b)} = \hat{C}_{MC} - b(\bar{X} - S_t),
$$
where $ \bar{X} = \frac{1}{n} \sum_{i=1}^{n} X_i $.

---

### Optimal Control Coefficient

The optimal $b$ minimizing the variance of the estimator is given by:
$$
b^* = \frac{\text{Cov}(A_i, X_i)}{\text{Var}(X_i)}
$$


In [6]:
def MC2_call(S0: float, K: float, sigma: float, r: float, T: float, N: int, steps: int) -> float:
    """
    Monte Carlo pricing of a European call option under GBM with exact law + cntrol variate

    Parameters
    ----------
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    sigma : float
        Volatility of the underlying (annualized).
    r : float
        Risk-free interest rate (annualized, continuously compounded).
    T : float
        Time to maturity in years.
    N : int
        Number of simulated paths.
    steps : int
        Number of time steps per path.

    Returns
    -------
    float
        Monte Carlo estimate of the European call price.
    """
    dt = T / steps

    # Draw standard normals: shape (N, steps)
    Z = np.random.randn(N, steps)

    # Log-returns per step
    dlogS = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z

    # Sum over time to get log S_T for each path
    log_ST = np.log(S0) + dlogS.cumsum(axis=1)[:, -1]

    ST = np.exp(log_ST)

    # Control variate variable
    X = np.exp(-r * T) * ST

    # Discounted payoff
    payoffs = np.exp(-r * T) * np.maximum(ST - K, 0.0)

    # Control coef
    b = np.cov(X, payoffs, ddof=1)[0, 1] / np.var(X, ddof=1)
    
    return float(payoffs.mean() - b * (X.mean() - S0))

# Application
call_MC2 = MC2_call(S0, K, sigma, r, T, N, steps)
print('Prix de l\'option MC2 :', call_MC2)

Prix de l'option MC2 : 8.931599849618033


## 2.3 Euler scheme of $(S_t)_{t\geq0}$

Under $\mathbb{Q}$, the dynamic of $S$ is :

$$
dS_t = r S_t \, dt + \sigma S_t \, dW_t
$$


> **Euler scheme :** 

We here consider a time grid $0=t_0 < t_1 < \dots < t_n = T$, and for $i \in \{1, \dots, n-1\}$.
From the previous formula we have :

$$
S_{t_{i+1}} = S_{t_i} + r \int_{t_{i}}^{t_{i+1}} S_u du + \sigma \int_{t_{i}}^{t_{i+1}}  S_u \, dW_u
$$

We can approximate the two integrals by :

- $\int_{t_{i}}^{t_{i+1}} S_u du = S_{t_{i}} (t_{i+1} -t_{i}) + O_{\mathbb Q}(\Delta t^{3/2})$

- $\int_{t_{i}}^{t_{i+1}} S_u dWu = S_{t_{i}} (W_{i+1} -W_{i}) + O_{\mathbb Q}(\Delta t)$



---

the dynamic of $S$ can be approximate by :

$$
\boxed{S_{t_{i+1}} = S_{t_{i}} + r S_{t_{i}} (t_{i+1} -t_{i}) + \sigma (W_{t_{i+1}} - W_{t_{i}}) + O_{\mathbb Q}(\Delta t)}
$$



---


In [7]:
def MC3_call(S0: float, K: float, sigma: float, r: float, T: float, N: int, steps: int) -> float:
    """
    Monte Carlo pricing of a European call option under GBM with Euler discretization.

    Parameters
    ----------
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    sigma : float
        Volatility of the underlying (annualized).
    r : float
        Risk-free interest rate (annualized, continuously compounded).
    T : float
        Time to maturity in years.
    N : int
        Number of simulated paths.
    steps : int
        Number of time steps per path.

    Returns
    -------
    float
        Monte Carlo estimate of the European call price.
    """
    dt = T / steps

    # Draw standard normals: shape (N, steps)
    Z = np.random.randn(N, steps)

    # Matrix of trajectories
    S = np.empty((N, steps + 1), dtype=float)
    S[:, 0] = S0

    # Euler : S_{t+dt} = S_t + r S_t dt + sigma S_t sqrt(dt) Z
    for k in range(steps):
        S[:, k+1] = S[:, k] + r * S[:, k] * dt + sigma * S[:, k] * np.sqrt(dt) * Z[:, k]

    ST = S[:, -1]
    payoffs = np.exp(-r * T) * np.maximum(ST - K, 0.0)
    return (payoffs.mean())

# Application
call_MC3 = MC3_call(S0, K, sigma, r, T, N, steps)
print('Prix de l\'option MC3 :', call_MC3)

Prix de l'option MC3 : 8.666347454109372


## 2.4 Second-order scheme for $(S_t)_{t \ge 0}$

Under $\mathbb{Q}$, the dynamic of $S$ is :

$$
dS_t = r S_t \, dt + \sigma S_t \, dW_t
$$

Integral form on $[t_i, t_{i+1}]$:

$$
S_{t_{i+1}} = S_{t_i} + r \int_{t_i}^{t_{i+1}} S_u \, du + \sigma \int_{t_i}^{t_{i+1}} S_u \, dW_u
$$

---

### Local expansion

For any $u \in [t_i, t_{i+1}]$, we have:

$$
S(u) = S(t_i) + r \int_{t_i}^{u} S_r \, dr + \sigma \int_{t_i}^{u} S_r \, dW_r
$$

We approximate $S_r \approx S_{t_i}$ inside the integrals:

$$
S(u) \approx S_{t_i} + r S_{t_i} \int_{t_i}^{u} dr + \sigma S_{t_i} \int_{t_i}^{u} dW_r
= S_{t_i} + r S_{t_i} (u - t_i) + \sigma S_{t_i} (W_u - W_{t_i})
$$

---


### General symbolic formula

Using the formal expansion:

$$
S_{t_{i+1}} 
= 
S_{t_i} 
+ r S_{t_i} \, \Delta t
+ \sigma S_{t_i} \, \Delta W
+ r^2 S_{t_i} \! \int_{t_i}^{t_{i+1}} \! \! \left( \int_{t_i}^{u} dr \right) du
+ r \sigma S_{t_i} \! \int_{t_i}^{t_{i+1}} \! \! \left( \int_{t_i}^{u} dr \right) dW_u
+ \sigma r S_{t_i} \! \int_{t_i}^{t_{i+1}} \! \! \left( \int_{t_i}^{u} dr \right) dW_u
+ \sigma^2 S_{t_i} \! \int_{t_i}^{t_{i+1}} \! \! \left( \int_{t_i}^{u} dW_r \right) dW_u
$$

---

### Useful identities

- $\boxed{\int_{t_i}^{t_{i+1}} (u - t_i)\, du = \tfrac{1}{2} (\Delta t)^2}$

- $\boxed{\int_{t_i}^{t_{i+1}} \int_{t_i}^{u} dW_r \, dW_u = \tfrac{1}{2} \left((\Delta W)^2 - \Delta t \right)}  \space \space{\scriptsize(\text{int: apply Itô to } (Y_u)^2 \text{ where } Y_u = W_u - W_{t_i})}$ 

Let $\Delta t = t_{i+1} - t_i$ and $\Delta W = W_{t_{i+1}} - W_{t_i}$.

---

> **First order approximation (Euler):**

$$
S_{t_{i+1}} \approx S_{t_i} + r S_{t_i} \, \Delta t + \sigma S_{t_i} \, \Delta W + O_{\mathbb{Q}}(\Delta t)
$$

> **Second order approximation:**

$$
S_{t_{i+1}} \approx S_{t_i}
+ r S_{t_i} \, \Delta t
+ \sigma S_{t_i} \, \Delta W
+ \tfrac{1}{2} r^2 S_{t_i} (\Delta t)^2
+ r \sigma S_{t_i} \, \Delta t \, \Delta W
+ \tfrac{1}{2} \sigma^2 S_{t_i} \big((\Delta W)^2 - \Delta t\big)
+ O_{\mathbb{Q}}(\Delta t^{2})
$$

In [8]:
def MC4_call(S0: float, K: float, sigma: float, r: float, T: float, N: int, steps: int) -> float:
    """
    Monte Carlo pricing of a European call option under GBM 
    with a second-order Euler / Itô–Taylor scheme.

    Parameters
    ----------
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    sigma : float
        Volatility of the underlying (annualized).
    r : float
        Risk-free interest rate (annualized, continuously compounded).
    T : float
        Time to maturity in years.
    N : int
        Number of simulated paths.
    steps : int
        Number of time steps per path.

    Returns
    -------
    float
        Monte Carlo estimate of the European call price (second-order scheme).
    """

    dt = T / steps
    sqrt_dt = np.sqrt(dt)

    # Draw standard normals: shape (N, steps)
    Z = np.random.randn(N, steps)

    # Matrix of trajectories
    S = np.empty((N, steps + 1), dtype=float)
    S[:, 0] = S0

    # Second-order Itô–Taylor for GBM:
    # S_{t+dt} ≈ S_t
    #   + r S_t dt
    #   + sigma S_t dW
    #   + 0.5 r^2 S_t dt^2
    #   + r sigma S_t dt dW
    #   + 0.5 sigma^2 S_t (dW^2 - dt)
    for k in range(steps):
        dW = sqrt_dt * Z[:, k]
        dW2_minus_dt = dW * dW - dt

        S_t = S[:, k]
        S[:, k+1] = (S_t
                     + r * S_t * dt
                     + sigma * S_t * dW
                     + 0.5 * (r**2) * S_t * (dt**2)
                     + r * sigma * S_t * dt * dW
                     + 0.5 * (sigma**2) * S_t * dW2_minus_dt)

    ST = S[:, -1]
    payoffs = np.exp(-r * T) * np.maximum(ST - K, 0.0)
    return payoffs.mean()

# Application 
call_MC4 = MC4_call(S0, K, sigma, r, T, N, steps)
print("Prix de l'option MC4 (2e ordre) :", call_MC4)

Prix de l'option MC4 (2e ordre) : 9.165279682929595


<a id='2'></a>
## 3. PDE 


Under the risk–neutral measure $\mathbb{Q}$, the asset price satisfies:

$$
dS_t = r S_t\,dt + \sigma S_t\,dW_t.
$$

>**EDP Black Scholes :**


The price of a European call option at time $t$ is given by the Feynman–Kac representation:

$$
C_t = \mathbb{E}_{\mathbb{Q}}\!\left[e^{-r(T-t)}(S_T - K)_+ \mid \mathcal{F}_t\right].
$$

Conditionally on $\mathcal{F}_t$, we have:

$$
S_T = S_t \exp\!\Big((r - \tfrac{1}{2}\sigma^2)(T-t) + \sigma(W_T - W_t)\Big),
$$

so that:

$$
C_t = \mathbb{E}_{\mathbb{Q}}\!\left[e^{-r(T-t)}(S_t e^{(r-\frac{\sigma^2}{2})(T-t)+\sigma(W_T - W_t)} - K)_+ \mid \mathcal{F}_t\right]
= C(t, S_t).
$$

By the **Weak Markov property**, the price can be expressed as a deterministic function $C(t, s)$ evaluated at $s = S_t$.


For $C(t, S_t) \in C^{1,2}$, Itô’s lemma gives:

$$
dC(t, S_t)
= \left(\partial_t C + r S_t\,\partial_s C + \tfrac{1}{2}\sigma^2 S_t^2\,\partial_{ss}C\right)dt
+ \sigma S_t\,\partial_s C\,dW_t.
$$

By no arbitrage, any tradable option must earn the risk-free rate once hedged. Equivalently, we can write the option’s dynamics as
$$
dC(t,S_t) = r\,C(t,S_t)\,dt \;+\; \text{(zero-mean martingale term)}.
\tag{2}
$$

Equality of drifts gives the **Black–Scholes PDE**:
$$
\boxed{
\partial_t C \;+\; r s\,\partial_s C \;+\; \tfrac12 \sigma^2 s^2\,\partial_{ss}C \;-\; r C \;=\; 0,
\qquad C(T,s)=(s-K)_+ .
}
$$
---

